In [89]:
import geopandas as gpd
import pandas as pd

## Open GIS Data

In [90]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Age data

In [168]:
acs_marriage = pd.read_csv(
    "data/ACSST5Y2023.S1201-2025-05-02T023159.csv", thousands=","
)

In [169]:
acs_marriage = acs_marriage.T.reset_index()
acs_marriage.columns = acs_marriage.iloc[0].str.lstrip()
acs_marriage = acs_marriage.iloc[1:]

In [170]:
acs_marriage = acs_marriage[
    [
        "Label (Grouping)",
        "Population 15 years and over",
        "Males 15 years and over",
        "Females 15 years and over",
    ]
]
acs_marriage = acs_marriage.loc[:, ~acs_marriage.columns.duplicated()].copy()
acs_marriage_never_married = acs_marriage[
    acs_marriage["Label (Grouping)"].str.split("!!").str[1] == "Never married"
]

In [171]:
acs_marriage_never_married["PERCENT"] = (
    acs_marriage_never_married["Population 15 years and over"]
    .str.replace("%", "")
    .astype(float)
)
acs_marriage_never_married["NAME"] = (
    acs_marriage_never_married["Label (Grouping)"].str.split("!!").str[0]
)

/tmp/ipykernel_35591/233647097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs_marriage_never_married['PERCENT'] = acs_marriage_never_married['Population 15 years and over'].str.replace("%","").astype(float)
/tmp/ipykernel_35591/233647097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs_marriage_never_married['NAME'] = acs_marriage_never_married['Label (Grouping)'].str.split("!!").str[0]


In [175]:
acs_marriage_df = acs_marriage_never_married[["NAME", "PERCENT"]]

## Combine Data

In [176]:
acs_marriage_gdf = states_df.merge(acs_marriage_df, on="NAME", how="left")

In [179]:
acs_marriage_gdf = acs_marriage_gdf[["geometry", "NAME", "PERCENT"]].dropna()

In [180]:
acs_marriage_gdf = acs_marriage_gdf.to_crs(9311)

In [181]:
acs_marriage_gdf.to_file("data/marriage.gpkg")